<a href="https://colab.research.google.com/github/Blurr99/tensorflow_deepLearning/blob/main/04_transfer_learning_in_tensorflow_part_1_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'efficientnet/tensorflow2/b0-feature-vector/1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F1912%2F2646%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240223%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240223T050430Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D861a2a23f38aeb631e4cb26e0223a51ae35f791a218dc76d70d3d911d561d4bdc19097983fac25f451ad8053a3193c72bd6efbdf1995b38861f795c8f9eee436494c160cfa9ac0fc8b865e64d912bb903e54d05347087034b8db2e5b17a2bb4d1311409e595ea76c835fdaf546b1b5f8e1c51d50b6a12dd68b8e00751fc900983c6b7e826c16a35d9b568a7dc2dd7a81cc92567a9d5dfaa3e30e63818facc2b7f2841228dcc9fb6ac7b83a4bb10af32f23ad47edda5c99b1c5735d4a7ea79d97250a5d673107973550ee724afb88ee316dfa8bbad2ce676fa3efa5006832eb7e4871a84784a3c6ce8687ef6cc046f977feb028335d1be649ea72a07e55abf419,resnet-v2/tensorflow2/50-feature-vector/1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F2393%2F3218%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240223%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240223T050430Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dabca387864915e6061553f3eea6a6a6ac501017859b484bf86e8a65f8285135f8e20324b9b66d8c7ea22df1d0c52d78e06b3baf288654617e5afb83b7bac182d91481c57eb43b5f8c1fb31f3f51d2a93c65c096b562e0c72f1b1a22cb8756b4f486650a46a3a3f73df5e33df1c5649f7a346818741faddc97aec3a4817091c110c1e68685360a7b1d92c97c8a3c90c23393e6cebedc8a9e88345cea0866042bcf7f3bf979ede36d3bbe2221b2b22a7024785ac92937e1ab7d320b78806cab5b07175798411f92d15af09bd89c5df1ea67ef0f29294ba1c5cbc21b5db7f8983daae60c5db0764b397c5d8894813dcfc77aa234e9f2350ae46f1b50b09714a28c9'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Tranfer Learning with TensorFlow Part 1: Feature Extraction

Transfer Learning is leveraging a working model's existing architecture and learned patterns for a custom problem.

There are 2 main benefits:
1. Can leverage an existing model's architecture that is proven to works on problems similar to the custom problem.
2. Can leverage a working neural architecture which has already learned data to our own, then those patterns can be adapted to the custom data.

In [ ]:
!nvidia-smi

## Downloading and exploring the data

In [ ]:
# Getting the data (10% of 10 food classes data)
import zipfile

# Download the data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

# Unzip the downloaded file
zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip")
zip_ref.extractall()
zip_ref.close()

In [ ]:
# How many images in each folder?
import os

for dirpath, dirnames, filenames in os.walk("10_food_classes_10_percent"):
    print(f"In {dirpath}, there are {len(dirnames)} directories and {len(filenames)} images")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random

def plot_random(target_dir, target_class):
    target_folder = target_dir + "/" + target_class

    random_image = random.sample(os.listdir(target_folder), 1)

    img = mpimg.imread(target_folder + "/" + random_image[0])
    plt.imshow(img)
    plt.axis(False)
    plt.title(target_class)

In [ ]:
plot_random("/kaggle/working/10_food_classes_10_percent/train", "chicken_curry")

## Creating data loaders (preparing the data)

Using the `ImageDataGenerator` class to load in the images into batches.

In [ ]:
# Setup data inputs
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (224,224)
BATCH_SIZE = 32

train_dir = "/kaggle/working/10_food_classes_10_percent/train"
test_dir = "/kaggle/working/10_food_classes_10_percent/test"

train_datagen = ImageDataGenerator(rescale = 1/255.)
test_datagen = ImageDataGenerator(rescale = 1/255.)

print("Training Images")
train_data_10_percent = train_datagen.flow_from_directory(train_dir,
                                                         target_size = IMAGE_SHAPE,
                                                         batch_size = BATCH_SIZE,
                                                         class_mode = "categorical")

print("Testing Images")
test_data_10_percent = test_datagen.flow_from_directory(test_dir,
                                                       target_size = IMAGE_SHAPE,
                                                        batch_size = BATCH_SIZE,
                                                        class_mode = "categorical")

## Setting up callbacks (things to run whilst the model trains)

Callbacks are extra functionalities that can be added to models to be performed during or after training. Some of the most popular callbacks are:
* Tracking experiments with the TensorBoard callback.
* Model checkpoint with the ModelCheckpoints callback.
* Stopping a model from training before it trains too long with the EarlyStopping callback.

In [ ]:
# Create TensorBoard callback
import datetime

def create_tensorboard_callback(dir_name, experiment_name):
    log_dir = dir_name +"/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir)
    print(f"Saving TensorBoard log files to: {log_dir}")
    return tensorboard_callback

## Creating models using TensorFlow Hub

Instead of using TensorFlow to create models layer by layer from scratch, using TensorFlowHub to get the model's layers.

Pretrained models can be accessed from: https://www.kaggle.com/models.

Using the following feature vector model: "https://www.kaggle.com/models/google/efficientnet/frameworks/TensorFlow1/variations/b0-feature-vector/versions/1", found after browsing tensorflowhub.

In [ ]:
# Comparing the following 2 models:
resnet_url = "https://kaggle.com/models/google/resnet-v2/frameworks/TensorFlow2/variations/50-feature-vector/versions/1"

efficientnet_url = "https://www.kaggle.com/models/google/efficientnet/frameworks/TensorFlow2/variations/b0-feature-vector/versions/1"

In [ ]:
# Import dependencies
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

In [ ]:
# Making a create_model() function to create a model from a URL
def create_model(model_URL, num_classes = 10):
    """
        Takes a TensorFlow Hub URL, and creates a Keras Sequential model with it.

        Args:
            model_URL (str): A TensorFlow Hub feature extraction URL.
            num_classes (int): Number of output neurons in the output layer,
            should be euqal to number of target classes (default = 10).

        Returns:
            An uncompiled Keras Sequential model with model_url as feature extraction layer
            and Dense output layer with num_classes output layer.
    """
    # Download the prettrained model and asve it as a Keras layer
    handle = model_URL
    feature_extraction_layer = hub.KerasLayer(handle = handle,
                                             trainable= False, # Freeze the already trained patterns
                                             name = "feature_extraction_layer",
                                             input_shape = IMAGE_SHAPE + (3,))

    # Create a model
    model = tf.keras.Sequential([
        feature_extraction_layer,
        layers.Dense(num_classes, activation = "softmax", name = "output_layer")
    ])

    return model

### Creating and testing ResNet TensorFlow Hub Feature Extraction model

In [ ]:
# Create resnet model
resnet_model = create_model(resnet_url,
                           num_classes = train_data_10_percent.num_classes)

In [ ]:
resnet_model.summary()

In [ ]:
# Compile the model
resnet_model.compile(loss = "categorical_crossentropy",
                    optimizer = tf.keras.optimizers.Adam(),
                    metrics = ["accuracy"])

In [ ]:
# Fitting the model
resnet_history = resnet_model.fit(train_data_10_percent,
                epochs = 5,
                steps_per_epoch = len(train_data_10_percent),
                validation_data = test_data_10_percent,
                validation_steps = len(test_data_10_percent),
                callbacks = [create_tensorboard_callback(dir_name = "tensorflow_hub",
                                                        experiment_name = "resnet50v2")])

The transfer learning frature extractor model out performed ALL of the previous models (substantially) and in a quicker time AND with only 10% of the training dataset.

In [ ]:
# Creating a function to plot the loss curves
def plot_loss_curves(history):
    """
        Returns separate loss curves for training and validation sets.

        Args:
            history: TensorFlow history object

        Returns:
            Plots of training/validation, loss and accuracy metrics
    """
    loss = history.history["loss"]
    val_loss = history.history["val_loss"]

    accuracy = history.history["accuracy"]
    val_accuracy = history.history["val_accuracy"]

    epochs = range(len(accuracy))

    plt.figure()
    plt.plot(epochs, loss, label = "Training loss")
    plt.plot(epochs, val_loss, label = "Validation loss")
    plt.title("Loss")
    plt.xlabel("Epochs")
    plt.legend()

    plt.figure()
    plt.plot(epochs, accuracy, label = "Training Accuracy")
    plt.plot(epochs, val_accuracy, label = "Validation Accuracy")
    plt.title("Accuracy")
    plt.xlabel("Epochs")
    plt.legend()

In [ ]:
plot_loss_curves(resnet_history)

### Creating and testing EfficientNetB0 TensorFlow Hub Feature Extraction model

In [ ]:
# Creating an efficientnet model
efficientNet_model = create_model(model_URL=efficientnet_url,
                                  num_classes=train_data_10_percent.num_classes)

In [ ]:
efficientNet_model.summary()

In [ ]:
# Compiling the model
efficientNet_model.compile(loss = "categorical_crossentropy",
                           optimizer = tf.keras.optimizers.Adam(),
                           metrics = ["accuracy"])

In [ ]:
# Fitting the model
efficient_history = efficientNet_model.fit(train_data_10_percent,
                                           epochs = 5,
                                           steps_per_epoch = len(train_data_10_percent),
                                           validation_data = test_data_10_percent,
                                           validation_steps = len(test_data_10_percent),
                                          callbacks= [create_tensorboard_callback("tensorflow_hub",
                                                                                 experiment_name="efficientnetb0v1")])

In [ ]:
plot_loss_curves(efficient_history)

## Different types of Transfer Learning

* "As is" transfer learning - using an existing model with no changes what so ever (e.g. using ImageNet model on 1000 ImageNet classes)
* "Feature Extraction" transfer learning - use the prelearned patterns of an existing model (e.g. EfficientNetB0 trained on ImageNet) and adjust the output layer for a custom problem (e.g. 1000 classes -> 10 classes of food)
* "Fine-Tuning" transfer learning - use the prelearned patterns of an existing model and fine turn many or all of the underlying layers (including new output layers)

In [ ]:
!pip install tensorboard

In [ ]:
# Upload TensorBoard dev records
import subprocess

# Define the TensorBoard Dev upload command
command = [
    "tensorboard",
    "dev",
    "upload",
    "--logdir",
    "./tensorflow_hub/",
    "--name",
    "EfficientNetB0 vs. ResNet50v2",
    "--description",
    "Comparing two different TF hub feature extraction model architectures using 10% of the training data",
    "--one_shot",
]

# Run the command and provide 'y' as input to confirm
process = subprocess.Popen(command, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
output, error = process.communicate(input='y')

# Print the output and error for debugging (optional)
print("Output:", output)
print("Error:", error)